In [73]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [74]:
import numpy as np
import pandas as pd
import scipy as sp
import mne
import keras
import matplotlib.pyplot as plt
import torch
import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM,Dense ,GRU,SimpleRNN,Conv1D,MaxPooling1D,Flatten , Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

tf.compat.v1.disable_eager_execution()

In [75]:
def PrePadding(data, seglength):
    pad = (-len(data)) % seglength
    l = np.concatenate((data,np.zeros((pad, len(data[0])))))
    return l

In [88]:
ARDF = pd.read_csv(r"C:\Users\admin\Desktop\test excel\LB.csv", header= None, skiprows=2)
AR = np.array(ARDF)
print(f"raw data shape: {np.shape(AR)}")
seglength = 36000
newar = PrePadding(AR, seglength)
IGEsegs = np.array(np.split(newar, seglength)).reshape(-1, seglength, 20)
print(f"segment list shape:{np.shape(IGEsegs)}")
IGE_labels = np.ones(np.shape(IGEsegs)[0]).reshape(-1)


raw data shape: (1881497, 20)
segment list shape:(53, 36000, 20)


In [81]:
ARDF = pd.read_csv(r"C:\Users\admin\Desktop\test excel\pnes.csv", header= None, skiprows=2)
AR = np.array(ARDF)
print(f"raw data shape: {np.shape(AR)}")
seglength = 36000
newar = PrePadding(AR, seglength)
PNESsegs = np.array(np.split(newar, seglength)).reshape(-1, seglength, 20)
print(f"segment list shape:{np.shape(PNESsegs)}")
PNES_labels = np.zeros(np.shape(PNESsegs)[0]).reshape(-1)


raw data shape: (1964747, 20)
segment list shape:(55, 36000, 20)


In [101]:
X = np.append(IGEsegs, PNESsegs, axis=0)
y = np.append(IGE_labels, PNES_labels)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)

In [132]:
inputs = tf.keras.Input(shape=(X_train.shape[1], 20))
expand_dims = tf.expand_dims(inputs, axis=2)
rnn = tf.keras.layers.SimpleRNN(256, return_sequences=True)(inputs)
flatten = tf.keras.layers.Flatten()(rnn)
outputs = tf.keras.layers.Dense(3, activation='softmax')(flatten)
RNN_model = tf.keras.Model(inputs=inputs, outputs=outputs)
RNN_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 36000, 20)]       0         
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 36000, 256)        70912     
                                                                 
 flatten_6 (Flatten)         (None, 9216000)           0         
                                                                 
 dense_6 (Dense)             (None, 3)                 27648003  
                                                                 
Total params: 27,718,915
Trainable params: 27,718,915
Non-trainable params: 0
_________________________________________________________________


In [133]:
RNN_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [134]:

history_RNN = RNN_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Train on 68 samples, validate on 18 samples
Epoch 1/50
68/68 [==============================] - ETA: 0s - loss: 174.9854 - accuracy: 0.2647   

c:\Users\admin\anaconda3\lib\site-packages\keras\engine\training_v1.py:2333: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


68/68 [==============================] - 182s 3s/sample - loss: 174.9854 - accuracy: 0.2647 - val_loss: 101.2273 - val_accuracy: 0.6111
Epoch 2/50
68/68 [==============================] - 204s 3s/sample - loss: 392.0991 - accuracy: 0.5735 - val_loss: 205.6927 - val_accuracy: 0.3889
Epoch 3/50
68/68 [==============================] - 207s 3s/sample - loss: 256.5256 - accuracy: 0.6176 - val_loss: 841.4971 - val_accuracy: 0.6111
Epoch 4/50


: 

: 